![KTS logo](https://raw.githubusercontent.com/konodyuk/kts/master/docs/static/banner_alpha.png)
# Modelling Guide

In [1]:
import numpy as np
np.random.seed(0)

import kts
from kts import *

Feature constructors and helpers defined earlier are automatically loaded:

In [2]:
simple_feature

simple_feature

Use `kts.ls` to list objects saved in your user cache and `kts.rm` to remove them:

In [3]:
print(kts.ls())
kts.rm('external')
print(kts.ls())

['train', 'test', 'external']
['train', 'test']


In [4]:
train = kts.load('train')
test = kts.load('test')

## Models

`kts.models.{binary, multiclass, regression}` contain most popular models for each task type.  
In particular, all of the corresponding sklearn models are present, as well as CatBoost, LGBM and XGB if already installed. We'll also add neural nets there soon.

In [5]:
from kts.models import binary, multiclass, regression

Init signatures are preserved:

In [6]:
cb = binary.CatBoostClassifier(iterations=100, rsm=.15, custom_metric='AUC')
cb

In [7]:
lr = binary.LogisticRegression(C=.5, solver='lbfgs', max_iter=1000)
lr

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [8]:
from category_encoders import TargetEncoder, WOEEncoder

fs = FeatureSet([simple_feature, interactions('Pclass', 'Age'), num_aggs('Fare'), tfidf('Name'), stl.one_hot_encode('Embarked')], 
                [stl.category_encode(TargetEncoder(), 'Embarked', 'Survived'), 
                 stl.category_encode(WOEEncoder(), 'Embarked', 'Survived')],
                train_frame=train,
                targets='Survived')

## Validation

To define a validation scheme, you'll use `kts.Validator(splitter, metric)`. Splitter is used to split the training set, and metric is for evaluating trained models.

In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

skf = StratifiedKFold(5, True, 42)
val = Validator(skf, roc_auc_score)

Running validation is as easy as `val.score(model, feature_set)`:

In [10]:
val.score(cb, fs)

{'score': 0.8450484134619144, 'id': 'KPBVAI'}

In [11]:
val.score(lr, fs)

{'score': 0.8145216602070352, 'id': 'FYCMDA'}

## Leaderboard

Right after validation your experiments are placed in the leaderboard:

In [12]:
lb

### Multiple Leaderboards

You can also keep multiple leaderboards by passing `leaderboard` parameter to `val.score()`. Default leaderboard is `main`.

In [13]:
some_model = binary.KNeighborsClassifier()

val.score(some_model, fs, leaderboard='other')

{'score': 0.7948359406496035, 'id': 'GWZCPQ'}

Use `kts.leaderboard_list` or `kts.lbs` to access leaderboards other than main:

In [14]:
lbs.main is lb
lbs.other is lbs['other']
leaderboard_list is lbs

True

True

True

Note that the new experiment appeared only in the new leaderboard:

In [15]:
lb
lbs.other

## Experiments

Experiments are accessible by their identifiers:

In [16]:
lb['KPBVAI'] is lb.KPBVAI

True

In [17]:
lb.KPBVAI

### Inference

Inference is as easy as `experiment.predict(frame)`. Features are computed automatically.

In [18]:
lb.KPBVAI.predict(test.head(5))

array([0.17121523, 0.57615125, 0.0999157 , 0.23952768, 0.78401192])

### Feature Importances

In [19]:
lb.KPBVAI.feature_importances

<bound method Experiment.feature_importances of <kts.validation.experiment.Experiment object at 0x1c1de3e80>>

In [20]:
lb.KPBVAI.feature_importances(sort_by='mean', n_best=7)

Use `plot=False` to get feature importances by fold:

In [21]:
lb.KPBVAI.feature_importances(plot=False)

is_male tfidf__Name_2 Embarked_ce_Survived_TargetEncoder Pclass_add_Age  \
0  39.5066       5.48229                             1.1201        1.96158   
1  34.2684        3.8706                            1.37644         2.7143   
2  32.5297       5.13815                            1.76954        5.26318   
3  30.0568       5.76845                            1.03283        2.74786   
4  51.2338       2.74165                            2.91524       0.848658   

  Pclass_sub_Age tfidf__Name_1 tfidf__Name_3 Embarked_ce_OneHotEncoder_3  \
0        6.07287       3.16809       5.17552                           0   
1        5.58991       3.79159       3.33875                           0   
2        6.86313       4.45559        3.8642                           0   
3        11.0083        6.2502       4.34941                           0   
4         2.5172       4.45662      0.492911                           0   

  Fare_div_std tfidf__Name_0 Fare_sub_div_mean tfidf__Name_4  \
0      3.48855       2.94677           7.29455       5.95125   
1      2.16574       5.99983           12.0095       4.67407   
2       5.2687       2.48002           5.46335       4.27618   
3      6.06301       4.31537           3.78222       5.14516   
4     0.428414       1.74347          0.741891       4.42125   

  Embarked_ce_OneHotEncoder_0 Embarked_ce_OneHotEncoder_1  \
0                     2.87565                      2.0481   
1                     3.57352                     2.55421   
2                     2.11486                     1.56882   
3                    0.910795                     1.48347   
4                           0                     1.88876   

  Embarked_ce_OneHotEncoder_2 Embarked_ce_Survived_WOEEncoder Pclass_mul_Age  \
0                     1.83414                         1.27512        5.79585   
1                     1.11964                         1.27202        10.2945   
2                    0.846825                         2.98447        8.55423   
3                     1.30387                         1.87028        7.67064   
4                     0.30458                        0.262854        11.8818   

  Fare_div_mean  
0       4.00294  
1       1.38701  
2       6.55907  
3        6.2413  
4       13.1209

Specify an importance estimator to compute permutation importance:

In [22]:
lb.KPBVAI.feature_importances(sort_by='mean', estimator=Permutation(train, n_iters=10))

## Custom Models

Suppose you want to use some model which is not in `kts.models`, like [Regularized Greedy Forest](https://github.com/RGF-team/rgf/tree/master/python-package).

In [23]:
!pip3 install rgf_python > /dev/null

In [24]:
from rgf.sklearn import RGFClassifier

To use it, you simply need to create a class derived from both your classifier and `kts.CustomModel`. It may optionally include `preprocess` method or inherit it from some mixin, like `kts.NormalizeFillNAMixin`:

```python
class KTSWrapper(kts.CustomModel, somelib.SomeClassifier):
    ignored_params = [...]
    
    def preprocess(X, y=None):
        if y is None:
            print('if y is None then .predict is called')
        else:
            print('otherwise .fit')
        return X, y
```

An alternative approach is using `kts.custom_model(ModelClass, ignored_params, normalize_fillna=True/False)` function:
```python
RGF = custom_model(RGFClassifier, ignored_params=['memory_policy', 'n_jobs', 'verbose'], normalize_fillna=True)
```
However, subclassing gives more freedom in defining custom preprocessing.

In this example the classifier can't deal with NaN values, and we use `kts.NormalizeFillNAMixin` to add preprocessing method:

In [25]:
class RGF(NormalizeFillNAMixin, CustomModel, RGFClassifier):
    ignored_params = ['memory_policy', 'n_jobs', 'verbose']

In [26]:
RGF()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/rgf/utils.py:225: UserWarning: Cannot find FastRGF executable files. FastRGF estimators will be unavailable for usage.
  warnings.warn("Cannot find FastRGF executable files. "


RGF(algorithm='RGF', calc_prob='sigmoid', init_model=None, l2=0.1,
  learning_rate=0.5, loss='Log', max_leaf=1000, memory_policy='generous',
  min_samples_leaf=10, n_iter=None, n_jobs=-1, n_tree_search=1,
  normalize=False, opt_interval=100, reg_depth=1.0, sl2=None,
  test_interval=100, verbose=0)

In [27]:
val.score(RGF(), fs)

{'score': 0.7704334423329472, 'id': 'BFELLE'}

## Custom Validators

*TODO*